In [18]:
import pandas as pd
import xml.etree.ElementTree as ET
import glob
import os

In [19]:
import re

In [20]:
files = glob.glob('DATA/search_data/*.xml')

if not os.path.exists('DATA/publications'):
    os.makedirs('DATA/publications')

In [21]:
for path in files:
    
    sid, = re.findall(r'\\(\d+).xml', path)
    
    if os.path.exists(f'DATA/publications/{sid}.csv'):continue
    xmldoc = ET.parse(path)

    root = xmldoc.getroot()

    Series = []
    for DocSum in root.findall('DocSum'):
        data = {}
        for Item in DocSum:
            if Item.tag == 'Id':
                data['PMID'] = int(Item.text)
                continue

            name = Item.attrib['Name']
            if name in ('Source', 'DOI', 'PubDate', 'FullJournalName'):
                data[name] = Item.text

        Series.append(pd.Series(data))
    
    if not Series: continue
    df = pd.concat(Series, axis=1, sort=False).T
    df['sid'] = int(sid)
    
    df.to_csv(f'DATA/publications/{sid}.csv')